In [17]:
import pandas as pd
import numpy as np

DEBG = True
exclude_memb = 1 

if not DEBG:
  if len(sys.argv) < 2:
    print ("ERR_MISSING_ARG:   The column number in fort.38 is the sole and required argument of mostocc.py")
    print ("                   To view the file header & select the right column, type:  head -1 fort.38")
    print ("Examples:")
    print ("Header: ex   6  4  2  0  -2  -4  -6  -8  -10  -12  -14  -16  -18  -20  -22")
    print (" Col #:  1   2  3  4  5   6   7   8   9   10   11   12   13   14...")
    #    col index:  0   1  2  3      ...
    sys.exit(0)
  column_in = int(sys.argv[1])-1
else:
  column_in = 3-1
    
# Get occupancy from fort.38 column corresponding to input (col number)
try:
    occ_df = pd.read_csv("fort.38", sep="\s+", header = 0, usecols=[0, column_in], names=['conf', 'occ'] )

except:
    print ("Could not read occ file; Given column was: %d", column_in + 1)
    
occ_df.insert(0, 'res', occ_df['conf'].str.extract('([A-Z][0-9]{4}?)', expand=True), allow_duplicates=False)
occ_df.head(n=5)
#print(occ_df.columns.values.tolist())        # = ['res', 'conf', 'occ']

,res,conf,occ
0,A0017,ARG01A0017_001,0.0
1,A0017,ARG01A0017_002,0.0
2,A0017,ARG01A0017_003,0.0
3,A0017,ARG01A0017_004,0.0
4,A0017,ARG01A0017_005,0.0


### Testing various 'grouby' aggregation: 

In [18]:
#occ_df_byres_max = occ_df.groupby([ 'res']).max()   

##		conf			occ
##res		
##A0017	ARG03A0017_126	0.319
##A0018	ARG03A0018_102	0.685

##occ_df_byres_max['conf'].head()
## res is now the index, so always printed:
##res
##A0017    ARG03A0017_126
##...
#Name: conf, dtype: object

#occ_df_byres_max = occ_df.groupby([ 'res']).max().occ
##res
##A0017    0.319
##A0018    0.685

occ_df_byres_max = occ_df.groupby([ 'res'])['conf'].max() ### OK for the mostocc pupose ###
##res
##A0017    ARG03A0017_126
##A0018    ARG03A0018_102

#occ_df_byres_max = occ_df.groupby([ 'occ']).max()  ### NOT the correct result ###
##   	res	    	conf
##occ		
##0.000	B0468	_CLDMB0468_002
##0.001	B0457	VAL01A0081_003

occ_df_byres_max[:2]
##	conf	occ
##res		
##A0017	ARG03A0017_126	0.319
##A0018	ARG03A0018_102	0.685


res
A0017    ARG03A0017_126
A0018    ARG03A0018_102
Name: conf, dtype: object

In [1]:
confs=occ_df_byres_max.conf
confs

NameError: name 'occ_df_byres_max' is not defined

In [16]:
# read in a file and keep only the most occupied confs
pdb = open("step2_out.pdb").readlines()
#.         10        20        30        40        50        60        70        80
#0123456789.123456789.123456789.123456789.123456789.123456789.123456789.123456789.123456789.
#ATOM  59063  CL  _CL A0466_001   4.701   7.631  -1.032   1.937      -1.000      -1O000M000
#ATOM      2  N   ARG P0017_000 -22.061 -15.228  28.007   1.500      -0.350      BK____M000

pdbo = open("mostocc.pdb", "w")
#print "Name of the file: ", pdbo.name


for line in pdb:
  if len(line) < 82: continue
  if line[80:90] == 'BK________': continue   # membrane
  if line[26] == ' ': iCode = '_'
  else: iCode = line[26]
  confID = line[17:20]+line[80:82]+line[21:26]+iCode+line[27:30]
  ## confID: ARGBKA0017_000
  ## ok:
  ## occ_df[occ_df['res'].isin(['A0148'])]
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  toDO:    
  if (confID in confs or confID[3:5] == 'BK'):
    print (line[:-1]), pdbo.name

KeyError: 'conf'

In [16]:
def within_5pct(value, max):
    return bool(value>=float(max*0.95))

data = occ_df.apply(within_5pct, max=occ_df_byres_max[:2])

777


TypeError: ("Could not operate 0.95 with block values can't multiply sequence by non-int of type 'float'", 'occurred at index extra')

In [ ]:
# read in a file and keep only the most occupied confs
pdb = open("step2_out.pdb").readlines()
#.         10        20        30        40        50        60        70        80
#0123456789.123456789.123456789.123456789.123456789.123456789.123456789.123456789.123456789.
#ATOM  59063  CL  _CL A0466_001   4.701   7.631  -1.032   1.937      -1.000      -1O000M000
#ATOM      2  N   ARG P0017_000 -22.061 -15.228  28.007   1.500      -0.350      BK____M000

pdbo = open("mostocc.pdb", "w")
#print "Name of the file: ", pdbo.name
confs=occ_df_byres_max['conf']
confs.head()

In [83]:
from pandas import *
d = {"my_label": Series(['A','B','A','C','D','D','E'])}
# print(type(d)): dict
df = DataFrame(d)
def as_perc(value, total):
    return value/float(total)

def get_count(values):
    return len(values)

df[df['my_label']=='D']

grouped_cnt_df = df.groupby("my_label").my_label.agg(get_count)
# The .agg() method here takes a function that is applied to all values of the groupby object.
data = grouped_cnt_df.apply(as_perc, total=df.my_label.count())
data

my_label
A    0.285714
B    0.142857
C    0.142857
D    0.285714
E    0.142857
Name: my_label, dtype: float64